In [ ]:
"""
export_harmonics_rf.py - Google Earth Engine NDVI Analysis with Random Forest
"""
import ee
import math
import datetime

# Initialize Earth Engine
try:
    ee.Initialize()
    print("Earth Engine initialized successfully")
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

class Config:
    # Time Parameters
    START_DATE = '2022-10-01'
    END_DATE = '2023-06-02'
    
    # Cloud Masking Parameters
    CLOUD_FILTER = 60
    CLD_PRB_THRESH = 50
    NIR_DRK_THRESH = 0.15
    CLD_PRJ_DIST = 1
    BUFFER = 50
    
    # RF Parameters
    NUM_TREES = 100
    RF_VARIABLES = ['B2', 'B3', 'B4', 'B8', 'NDVI', 't']
    
    # Area Parameters
    AOI_COUNTRY_CODE = 256
    CRS = 'EPSG:27700'
    
    # Export Parameters
    EXPORT_FOLDER = 'GEE_Exports'
    EXPORT_PREFIX = 'rf_ndvi_analysis_'

cfg = Config()

# Helper Functions
def print_info(collection, name):
    size = collection.size().getInfo()
    date_range = ee.Image(collection.first()).date().format('YYYY-MM-dd').getInfo()
    print(f"{name}: {size} images from {date_range}")

# Core Processing Functions 
def get_s2_collection(aoi, start_date, end_date):
    s2_sr = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', cfg.CLOUD_FILTER)))
    
    s2_cloudless = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))
    
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(
        primary=s2_sr,
        secondary=s2_cloudless,
        condition=ee.Filter.equals(
            leftField='system:index',
            rightField='system:index'
        )
    ))

def add_time_variables(img):
    date = ee.Date(img.get('system:time_start'))
    years = date.difference(ee.Date('1970-01-01'), 'year')
    return img.addBands(ee.Image(years).rename('t'))

def add_ndvi(img):
    return img.addBands(
        img.normalizedDifference(['B8', 'B4']).rename('NDVI')
    )

def prepare_training_data(collection):
    """Converts ImageCollection to FeatureCollection for ML"""
    return collection.map(lambda img: 
        img.select(cfg.RF_VARIABLES)
        .sample(region=aoi, scale=10, numPixels=500)
    ).flatten()

# Main Processing Chain
def main():
    # 1. Set up Area of Interest
    aoi = ee.FeatureCollection('FAO/GAUL/2015/level0') \
            .filterMetadata('ADM0_CODE', 'equals', cfg.AOI_COUNTRY_CODE)
    print(f"AOI: {aoi.first().get('ADM0_NAME').getInfo()}")

    # 2. Get and preprocess Sentinel-2 data
    s2_col = get_s2_collection(aoi, cfg.START_DATE, cfg.END_DATE)
    processed_col = s2_col.map(add_time_variables).map(add_ndvi)
    print_info(processed_col, "Processed Collection")

    # 3. Prepare training data
    training_data = prepare_training_data(processed_col)
    print(f"Training samples: {training_data.size().getInfo()}")

    # 4. Train Random Forest model
    classifier = ee.Classifier.smileRandomForest(cfg.NUM_TREES) \
        .train(
            features=training_data,
            classProperty='NDVI',  # Regression mode
            inputProperties=cfg.RF_VARIABLES
        )

    # 5. Apply model to entire collection
    classified_col = processed_col.map(lambda img: 
        img.addBands(
            img.classify(classifier, 'rf_prediction')
    )

    # 6. Create time series predictions
    rf_median = classified_col.select('rf_prediction').median()

    # 7. Export results
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    task = ee.batch.Export.image.toDrive(
        image=rf_median.clip(aoi),
        description='RF_NDVI_Prediction',
        folder=cfg.EXPORT_FOLDER,
        fileNamePrefix=f"{cfg.EXPORT_PREFIX}{timestamp}",
        region=aoi.geometry(),
        scale=10,
        crs=cfg.CRS,
        maxPixels=1e13
    )
    task.start()
    print(f"Export started with Task ID: {task.id}")
    print(f"Monitor at: https://code.earthengine.google.com/tasks")

if __name__ == "__main__":
    main()